# Evaluation

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import regex as re

In [2]:
# Nhập dữ liệu
with open('seq.out', 'r', encoding= 'utf-8') as file:
    out_lines = file.readlines()
    file.close()

with open('seq_test.out', 'r', encoding= 'utf-8') as file:
    test_lines = file.readlines()
    file.close()

In [8]:
# Số samples trong bộ test
len(out_lines), len(test_lines)

(893, 893)

In [9]:
# Chuẩn hóa
out_lines = [x.strip() for x in out_lines]
test_lines = [x.strip() for x in test_lines]

In [10]:
out_lines[0]

'O O O O O O O B-fromloc.city_name O B-toloc.city_name O O O O O O B-stoploc.city_name'

In [11]:
test_lines[0]

'<flight> -> tôi muốn tìm một chuyến bay từ [đà_nẵng:B-fromloc.city_name] đến [phú_quốc:B-toloc.city_name] và có một trạm dừng ở [cam_ranh:B-stoploc.city_name]'

In [17]:
# Chuyển định dạng
def test2out(test_line):
    words = test_line.split()
    intent = words[0]
    words = words[2:]
    str = []
    for word in words:
        if word[0] != '[':
            str.append('O')
        else:
            info = word[1:-1]
            str.append(info.split(':')[-1])
    slots = ' '.join(str)
    return intent, slots

In [20]:
# Kiểm tra
intent, slots = test2out(test_lines[0])
slots

'O O O O O O O B-fromloc.city_name O B-toloc.city_name O O O O O O B-stoploc.city_name'

In [21]:
out_lines[0]

'O O O O O O O B-fromloc.city_name O B-toloc.city_name O O O O O O B-stoploc.city_name'

In [27]:
# Ghi file
outpath = 'slot_diff.txt'
lines = ['id||seq_test||seq_out\n']

for i in range(len(out_lines)):
    intent, slots = test2out(test_lines[i])
    if slots != out_lines[i]:
        lines.append('||'.join([str(i), slots, out_lines[i]]) + '\n')

with open(outpath, 'w', encoding= 'utf-8') as file:
    file.writelines(lines)
    file.close()

In [44]:
# Các lỗi gặp phải
diffs = []
for line in lines:
    words = [x.strip() for x in line.split('||')]
    test = words[1].split()
    out = words[2].split()
    
    for i in range(len(test)):
        if test[i] != out[i]:
            diffs.append(f'{test[i]}||{out[i]}')

diffs = {i:diffs.count(i) for i in set(diffs)}
diffs = dict(sorted(diffs.items(), key=lambda item: -item[1]))
diffs # test||out

{'B-toloc.city_name||B-city_name': 18,
 'I-fromloc.airport_name||I-airport_name': 18,
 'I-toloc.city_name||I-city_name': 12,
 'B-fromloc.airport_name||B-airport_name': 11,
 'B-city_name||B-state_name': 6,
 'B-flight_number||I-aircraft_code': 6,
 'B-aircraft_code||I-aircraft_code': 5,
 'O||B-flight_mod': 5,
 'B-flight_mod||I-flight_mod': 5,
 'B-fromloc.airport_code||B-airport_code': 4,
 'B-mod||O': 4,
 'B-fromloc.city_name||B-fromloc.airport_name': 4,
 'I-depart_time.time||I-arrive_time.time': 4,
 'B-fromloc.city_name||B-city_name': 4,
 'I-arrive_date.day_number||I-depart_date.day_number': 3,
 'I-city_name||B-state_name': 3,
 'B-toloc.city_name||B-toloc.state_name': 3,
 'B-fromloc.state_name||I-fromloc.state_name': 2,
 'B-airline_name||O': 2,
 'I-airline_name||B-airline_name': 2,
 'B-arrive_date.day_name||B-depart_date.day_name': 2,
 'B-depart_time.time||B-arrive_time.time': 2,
 'I-depart_date.day_number||I-arrive_date.day_number': 2,
 'B-arrive_date.month_name||B-depart_date.month_name